In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Display options (optional)
pd.set_option('display.max_columns', 50)

BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/VolSurf_ML"
METRICS_DIR = os.path.join(BASE_DIR, "metrics")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [45]:
# ---- Load SVI metrics ----
svi_files = sorted(glob.glob(os.path.join(METRICS_DIR, "svi_metrics_*.parquet")))
print("SVI files:")
for f in svi_files:
    print("  ", os.path.basename(f))

if not svi_files:
    raise RuntimeError("No svi_metrics_*.parquet files found in METRICS_DIR.")

svi_dfs = [pd.read_parquet(f) for f in svi_files]
svi_all = pd.concat(svi_dfs, ignore_index=True)

# Normalise types
svi_all['asof']   = pd.to_datetime(svi_all['asof'])
svi_all['expiry'] = pd.to_datetime(svi_all['expiry'])

# ---- Load SSVI metrics ----
ssvi_files = sorted(glob.glob(os.path.join(METRICS_DIR, "ssvi_metrics_*.parquet")))
print("\nSSVI files:")
for f in ssvi_files:
    print("  ", os.path.basename(f))

if not ssvi_files:
    raise RuntimeError("No ssvi_metrics_*.parquet files found in METRICS_DIR.")

ssvi_dfs = [pd.read_parquet(f) for f in ssvi_files]
ssvi_all = pd.concat(ssvi_dfs, ignore_index=True)

ssvi_all['asof']   = pd.to_datetime(ssvi_all['asof'])
ssvi_all['expiry'] = pd.to_datetime(ssvi_all['expiry'])

# ---- Merge SVI + SSVI on (asof, expiry, T) ----
# Keep only SSVI columns we care about
ssvi_cols = ['asof','expiry','T','theta','ssvi_rho','ssvi_eta','ssvi_p','fit_cost']
if 'calendar_violations' in ssvi_all.columns:
    ssvi_cols.append('calendar_violations')

metrics_all = svi_all.merge(
    ssvi_all[ssvi_cols],
    on=['asof','expiry'],
    how='left',
    suffixes=('', '_ssvi')
)

metrics_all = metrics_all.sort_values(['asof', 'T']).reset_index(drop=True)

print("\nCombined metrics shape:", metrics_all.shape)
metrics_all.head()

SVI files:
   svi_metrics_2025-11-15.parquet

SSVI files:
   ssvi_metrics_2025-11-15.parquet

Combined metrics shape: (12, 18)


,asof,expiry,T,ATM_IV,ATM_skew,ATM_curvature,a,b,rho,m,sigma,T_ssvi,theta,ssvi_rho,ssvi_eta,ssvi_p,fit_cost,calendar_violations
0,2025-11-15,2025-11-21,0.016438,3.662370,-4.385685,115.634223,-200.404446,84.841837,-0.334934,-0.873461,2.509585,0.016427,0.001134,-0.406584,1.389818,0.632314,2.621373e-02,101.0
1,2025-11-15,2025-11-28,0.035616,1.918499,0.788328,48.930020,-21.518208,34.778988,-0.711780,-0.906005,0.886200,0.035592,0.001930,-0.333633,1.285469,0.549368,3.117243e-04,0.0
2,2025-11-15,2025-12-12,0.073973,1.260399,-2.288092,12.166127,-0.260319,8.770027,0.878415,0.215133,0.086999,0.073922,0.004252,-0.222240,0.799124,0.515132,9.165372e-06,0.0
3,2025-11-15,2025-12-19,0.093151,1.209491,2.968839,20.145789,-19.744983,18.566359,0.489296,0.623687,1.226523,0.093087,0.005366,-0.343621,1.531057,0.579717,7.114869e-02,110.0
4,2025-11-15,2025-12-26,0.112329,1.010595,-1.278680,9.367754,0.079509,0.401723,-0.395158,0.025417,0.073330,0.112252,0.006502,-0.299437,0.619621,0.504799,9.102784e-08,0.0


In [47]:
price_path = os.path.join(BASE_DIR, "hist_price.parquet")
prices = pd.read_parquet(price_path)

# Ensure index is sorted
prices = prices.sort_index()

# If index is timezone-aware (like 2023-11-15 00:00:00-05:00), drop tz
if hasattr(prices.index, "tz") and prices.index.tz is not None:
    # Option 1: convert to UTC then drop tz
    prices.index = prices.index.tz_convert("UTC").tz_localize(None)
    # Option 2 (simpler, also fine here): prices.index = prices.index.tz_localize(None)

# Basic returns and realised vol
prices['return_1d'] = prices['Close'].pct_change()
prices['return_5d'] = prices['Close'].pct_change(5)
prices['rv_20d']    = prices['return_1d'].rolling(20).std() * np.sqrt(252)

# Turn index into a column called 'asof' (date-like, no tz)
prices = prices.reset_index().rename(columns={prices.index.name or 'Date': 'asof'})
prices['asof'] = pd.to_datetime(prices['asof']).dt.normalize()

prices.head()

,asof,Open,High,Low,Close,Volume,Dividends,Stock Splits,return_1d,return_5d,rv_20d
0,2023-11-15,141.991727,143.617657,141.991727,143.215958,8365300,0.0,0.0,NaN,NaN,NaN
1,2023-11-16,143.550709,144.927972,143.550709,144.851456,8126200,0.0,0.0,0.011420,NaN,NaN
2,2023-11-17,145.568792,146.200040,144.937544,146.161789,7812400,0.0,0.0,0.009046,NaN,NaN
3,2023-11-20,145.454024,147.051262,145.300992,146.611298,7019000,0.0,0.0,0.003075,NaN,NaN
4,2023-11-21,146.525227,146.936500,145.922686,146.305252,7083500,0.0,0.0,-0.002087,NaN,NaN


In [48]:
df = metrics_all.merge(
    prices[['asof', 'Close', 'return_1d', 'return_5d', 'rv_20d']],
    on='asof',
    how='left'
)

df = df.rename(columns={'Close': 'spot'})
df = df.sort_values(['T', 'asof']).reset_index(drop=True)

print("Merged shape:", df.shape)
df.head()


Merged shape: (12, 22)


,asof,expiry,T,ATM_IV,ATM_skew,ATM_curvature,a,b,rho,m,sigma,T_ssvi,theta,ssvi_rho,ssvi_eta,ssvi_p,fit_cost,calendar_violations,spot,return_1d,return_5d,rv_20d
0,2025-11-15,2025-11-21,0.016438,3.662370,-4.385685,115.634223,-200.404446,84.841837,-0.334934,-0.873461,2.509585,0.016427,0.001134,-0.406584,1.389818,0.632314,2.621373e-02,101.0,NaN,NaN,NaN,NaN
1,2025-11-15,2025-11-28,0.035616,1.918499,0.788328,48.930020,-21.518208,34.778988,-0.711780,-0.906005,0.886200,0.035592,0.001930,-0.333633,1.285469,0.549368,3.117243e-04,0.0,NaN,NaN,NaN,NaN
2,2025-11-15,2025-12-12,0.073973,1.260399,-2.288092,12.166127,-0.260319,8.770027,0.878415,0.215133,0.086999,0.073922,0.004252,-0.222240,0.799124,0.515132,9.165372e-06,0.0,NaN,NaN,NaN,NaN
3,2025-11-15,2025-12-19,0.093151,1.209491,2.968839,20.145789,-19.744983,18.566359,0.489296,0.623687,1.226523,0.093087,0.005366,-0.343621,1.531057,0.579717,7.114869e-02,110.0,NaN,NaN,NaN,NaN
4,2025-11-15,2025-12-26,0.112329,1.010595,-1.278680,9.367754,0.079509,0.401723,-0.395158,0.025417,0.073330,0.112252,0.006502,-0.299437,0.619621,0.504799,9.102784e-08,0.0,NaN,NaN,NaN,NaN


In [49]:
df = df.sort_values(['T', 'asof']).reset_index(drop=True)

# Shift ATM_IV one day ahead within each tenor
df['ATM_IV_tplus1'] = df.groupby('T')['ATM_IV'].shift(-1)

# Drop rows where we don't have next-day target
df = df.dropna(subset=['ATM_IV_tplus1']).reset_index(drop=True)

print("After creating target, shape:", df.shape)
df[['asof', 'T', 'ATM_IV', 'ATM_IV_tplus1']].head(10)


After creating target, shape: (0, 23)


,asof,T,ATM_IV,ATM_IV_tplus1


In [38]:
feature_cols = [
    'T',
    # ATM + SVI
    'ATM_IV', 'ATM_skew', 'ATM_curvature',
    'a', 'b', 'rho', 'm', 'sigma',
    # SSVI
    'theta', 'ssvi_rho', 'ssvi_eta', 'ssvi_p',
    # optional: calendar / fit diagnostics
    # 'fit_cost',
    # 'calendar_violations',
    # market features (when merge_asof is fixed)
    # 'return_1d', 'return_5d', 'rv_20d', 'spot',
]

target_col = 'ATM_IV_tplus1'

data = df.dropna(subset=feature_cols + [target_col]).copy()
data = data.sort_values('asof').reset_index(drop=True)

print("Final usable rows:", len(data))
data[ ['asof', 'T', 'ATM_IV', 'ATM_IV_tplus1'] + feature_cols ].head()


Final usable rows: 0


,asof,T,ATM_IV,ATM_IV_tplus1,T,ATM_IV,ATM_skew,ATM_curvature,a,b,rho,m,sigma,theta,ssvi_rho,ssvi_eta,ssvi_p


In [39]:
n = len(data)
if n < 30:
    print("⚠️ Warning: very few samples for ML. The notebook will run, but forecasting quality will be limited until you collect more days.")

train_end = int(n * 0.7)
val_end   = int(n * 0.85)

train = data.iloc[:train_end]
val   = data.iloc[train_end:val_end]
test  = data.iloc[val_end:]

print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

X_train = train[feature_cols].values.astype(np.float32)
y_train = train[target_col].values.astype(np.float32)

X_val   = val[feature_cols].values.astype(np.float32)
y_val   = val[target_col].values.astype(np.float32)

X_test  = test[feature_cols].values.astype(np.float32)
y_test  = test[target_col].values.astype(np.float32)

dates_test = test['asof'].values


⚠️ Warning: very few samples for ML. The notebook will run, but forecasting quality will be limited until you collect more days.
Train: 0, Val: 0, Test: 0


In [40]:
if len(test) > 0:
    y_pred_persist = test['ATM_IV'].values.astype(np.float32)

    mae_persist = mean_absolute_error(y_test, y_pred_persist)
    rmse_persist = mean_squared_error(y_test, y_pred_persist, squared=False)
    print(f"Baseline (persistence) – MAE: {mae_persist:.6f}, RMSE: {rmse_persist:.6f}")
else:
    print("No test set; cannot compute baseline yet.")


No test set; cannot compute baseline yet.


In [41]:
def make_loader(X, y, batch_size=64, shuffle=False):
    X_tensor = torch.from_numpy(X)
    y_tensor = torch.from_numpy(y).unsqueeze(1)  # (N,) -> (N,1)
    ds = TensorDataset(X_tensor, y_tensor)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = make_loader(X_train, y_train, batch_size=64, shuffle=True)
val_loader   = make_loader(X_val,   y_val,   batch_size=256, shuffle=False)
test_loader  = make_loader(X_test,  y_test,  batch_size=256, shuffle=False)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [42]:
input_dim = X_train.shape[1]

class VolForecastNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, 1)
        self.act = nn.ReLU()

    def forward(self, x):
        # x shape: (batch_size, input_dim)

        # Layer 1: Linear -> BatchNorm -> ReLU
        x = self.fc1(x)
        x = self.bn1(x)   # <-- BatchNorm applied here
        x = self.act(x)

        # Layer 2: Linear -> BatchNorm -> ReLU
        x = self.fc2(x)
        x = self.bn2(x)   # <-- BatchNorm applied here
        x = self.act(x)

        # Output layer: Linear (no BatchNorm, no activation)
        x = self.fc3(x)
        return x

model = VolForecastNet(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model


VolForecastNet(
  (fc1): Linear(in_features=13, out_features=64, bias=True)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (act): ReLU()
)

In [15]:
best_val_loss = float('inf')
best_state = None
patience = 10
patience_counter = 0

train_history = []
val_history = []

for epoch in range(100):  # max epochs
    # ---- Train ----
    model.train()
    batch_losses = []
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())

    train_loss = np.mean(batch_losses)

    # ---- Validate ----
    model.eval()
    val_losses = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses) if val_losses else np.nan

    train_history.append(train_loss)
    val_history.append(val_loss)

    print(f"Epoch {epoch:03d} | train {train_loss:.6f} | val {val_loss:.6f}")

    # Early stopping
    if val_losses and val_loss < best_val_loss - 1e-6:
        best_val_loss = val_loss
        best_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping.")
            break

# Load best model (if we have one)
if best_state is not None:
    model.load_state_dict(best_state)


NameError: name 'train_loader' is not defined

In [16]:
model.eval()
y_pred_list = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy().ravel()
        y_pred_list.append(preds)

if y_pred_list:
    y_pred_nn = np.concatenate(y_pred_list)

    mae_nn = mean_absolute_error(y_test, y_pred_nn)
    rmse_nn = mean_squared_error(y_test, y_pred_nn, squared=False)

    print(f"PyTorch MLP – MAE: {mae_nn:.6f}, RMSE: {rmse_nn:.6f}")
    if len(test) > 0:
        print(f"Baseline     – MAE: {mae_persist:.6f}, RMSE: {rmse_persist:.6f}")
else:
    print("No predictions from test_loader (probably empty test set).")


NameError: name 'test_loader' is not defined

In [17]:
if len(test) > 0 and y_pred_list:
    plt.figure(figsize=(12,5))
    plt.plot(dates_test, y_test, label='Actual ATM_IV_{t+1}', marker='o')
    plt.plot(dates_test, y_pred_persist, label='Persistence', linestyle='--')
    plt.plot(dates_test, y_pred_nn, label='PyTorch MLP', linestyle='-.')
    plt.xlabel("Date")
    plt.ylabel("ATM_IV_{t+1}")
    plt.title("Next-day ATM IV Forecast – Actual vs Models")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Not enough test data to plot.")


Not enough test data to plot.
